### Machine Learning Homework
### Problem Set 3
### Gaurav Shah

In [38]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras import layers
from keras import regularizers
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_excel('C:\\Users\\shahg_000\\Downloads\\data.xlsx')
df=data.set_index('Date')

In [4]:
TrainingSet=df.loc[:'2002-12-31',:]
TestSet=df.loc['2003-01-01':,:]

In [5]:
def Stacking(toBeStacked,p):
    n=p+1
    stackedF=pd.DataFrame()
    temp=np.zeros(shape=(1,n))
    for cnum in range(toBeStacked.shape[1]):
        temp1=np.zeros(shape=(1,n))
        for rnum in range(n,len(toBeStacked)+1):
            temp1=np.append(temp1,[np.flip(np.array(toBeStacked.iloc[rnum-n:rnum,cnum]))],axis=0)
        temp1=np.delete(temp1,(0),axis=0)
        temp=np.append(temp,temp1,axis=0)
    return(pd.DataFrame(np.delete(temp,(0),axis=0)))

In [6]:
def DummyVariables(toBeDummy):
    p=int(toBeDummy.shape[0]/30)
    finalDataframe = pd.DataFrame()
    for i in range(0,29):
        dummyframes = pd.DataFrame(data = 0, index=range(p),columns=range(28))
        dummyframes.insert(i, "DV"+str(i), 1)
        finalDataframe = finalDataframe.append(dummyframes)
    dvAllZero = pd.DataFrame(data = 0, index=range(p),columns=range(28))
    finalDataframe = finalDataframe.append(dvAllZero, ignore_index=True)
    finalDataframe.drop(finalDataframe.iloc[:, 0:28], inplace = True, axis = 1) 
    finalDataframe = finalDataframe.fillna(0)
    return(pd.concat([toBeDummy, finalDataframe], axis=1))

In [7]:
%%time
TrainingStacked=Stacking(TrainingSet,30)
TestStacked=Stacking(TestSet,30)

Wall time: 1min 10s


In [8]:
TrainingDummyStacked=DummyVariables(TrainingStacked)
TestDummyStacked=DummyVariables(TestStacked)

C:\Users\shahg_000\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [9]:
TrainingDummyStacked.head()

,0,1,2,3,4,5,6,7,8,9,...,DV26,DV27,DV28,DV3,DV4,DV5,DV6,DV7,DV8,DV9
0,0.022162,0.008439,-0.014025,-0.022039,-0.008141,0.021859,0.039442,0.014472,0.017648,-0.017648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.016575,0.022162,0.008439,-0.014025,-0.022039,-0.008141,0.021859,0.039442,0.014472,0.017648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.011080,-0.016575,0.022162,0.008439,-0.014025,-0.022039,-0.008141,0.021859,0.039442,0.014472,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.011080,-0.016575,0.022162,0.008439,-0.014025,-0.022039,-0.008141,0.021859,0.039442,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.021799,0.000000,0.011080,-0.016575,0.022162,0.008439,-0.014025,-0.022039,-0.008141,0.021859,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
TrainingDummyStacked.describe()

,0,1,2,3,4,5,6,7,8,9,...,DV26,DV27,DV28,DV3,DV4,DV5,DV6,DV7,DV8,DV9
count,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,...,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000,118740.000000
mean,0.000511,0.000510,0.000505,0.000509,0.000504,0.000512,0.000512,0.000509,0.000515,0.000513,...,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333
std,0.020482,0.020484,0.020485,0.020484,0.020487,0.020492,0.020492,0.020491,0.020494,0.020497,...,0.179506,0.179506,0.179506,0.179506,0.179506,0.179506,0.179506,0.179506,0.179506,0.179506
min,-0.379490,-0.379490,-0.379490,-0.379490,-0.379490,-0.379490,-0.379490,-0.379490,-0.379490,-0.379490,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.010210,-0.010216,-0.010227,-0.010219,-0.010230,-0.010223,-0.010230,-0.010231,-0.010227,-0.010231,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.011233,0.011234,0.011227,0.011231,0.011232,0.011236,0.011236,0.011236,0.011243,0.011243,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.226528,0.226528,0.226528,0.226528,0.226528,0.226528,0.226528,0.226528,0.226528,0.226528,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
y=TrainingDummyStacked.loc[:,0]
X=TrainingDummyStacked.drop([0],axis=1)
y_test=TestDummyStacked.loc[:,0]
X_test=TestDummyStacked.drop([0],axis=1)

Q.1

In [12]:
model = Sequential()
model.add(Dense(30, activation='relu', input_shape=(59,)))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [14]:
early_stopping_monitor = EarlyStopping(patience=5)
model.fit(X,y, validation_split=0.2, epochs=30, 
          callbacks=[early_stopping_monitor],batch_size=20)

Train on 94992 samples, validate on 23748 samples
Epoch 1/30
94992/94992 [==============================] - 7s 74us/step - loss: 4.7347e-04 - val_loss: 3.5501e-04
Epoch 2/30
94992/94992 [==============================] - 7s 76us/step - loss: 4.4522e-04 - val_loss: 3.3159e-04
Epoch 3/30
94992/94992 [==============================] - 7s 69us/step - loss: 4.4419e-04 - val_loss: 3.2945e-04
Epoch 4/30
94992/94992 [==============================] - 7s 72us/step - loss: 4.4427e-04 - val_loss: 3.3201e-04
Epoch 5/30
94992/94992 [==============================] - 6s 67us/step - loss: 4.4306e-04 - val_loss: 3.2911e-04
Epoch 6/30
94992/94992 [==============================] - 7s 69us/step - loss: 4.4190e-04 - val_loss: 3.2883e-04
Epoch 7/30
94992/94992 [==============================] - 7s 74us/step - loss: 4.4157e-04 - val_loss: 3.2863e-04
Epoch 8/30
94992/94992 [==============================] - 7s 72us/step - loss: 4.4103e-04 - val_loss: 3.2944e-04
Epoch 9/30
94992/94992 [======================

In [15]:
y_pred_model1=model.predict(X_test)
print("The mean absoulte error is :",mean_absolute_error(y_test,y_pred_model1))

The mean absoulte error is : 0.012856670606135746


Q.2

In [34]:
model2 = Sequential()
model2.add(Dense(15, activation='relu', input_shape=(59,)))
model2.add(layers.Dropout(0.3, input_shape=(59,)))
model2.add(Dense(10, activation='relu'))
model2.add(layers.Dropout(0.3, input_shape=(59,)))
model2.add(Dense(5, activation='relu'))
model2.add(layers.Dropout(0.3, input_shape=(59,)))
model2.add(Dense(1))

In [35]:
model2.compile(optimizer='adam', loss='mean_squared_error')

In [37]:
model2.fit(X,y, epochs=30, callbacks=[early_stopping_monitor],batch_size=20)

Epoch 1/30
118740/118740 [==============================] - 10s 88us/step - loss: 4.4783e-04
Epoch 2/30
  1720/118740 [..............................] - ETA: 11s - loss: 4.1495e-04

C:\Users\shahg_000\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


118740/118740 [==============================] - 10s 83us/step - loss: 4.2186e-04
Epoch 3/30
118740/118740 [==============================] - 9s 76us/step - loss: 4.2170e-04
Epoch 4/30
118740/118740 [==============================] - 10s 84us/step - loss: 4.2185e-04
Epoch 5/30
118740/118740 [==============================] - 10s 82us/step - loss: 4.2164e-04
Epoch 6/30
118740/118740 [==============================] - 10s 83us/step - loss: 4.2173e-04
Epoch 7/30
118740/118740 [==============================] - 10s 88us/step - loss: 4.2176e-04
Epoch 8/30
118740/118740 [==============================] - 11s 92us/step - loss: 4.2197e-04
Epoch 9/30
118740/118740 [==============================] - 10s 83us/step - loss: 4.2163e-04
Epoch 10/30
118740/118740 [==============================] - 10s 87us/step - loss: 4.2185e-04
Epoch 11/30
118740/118740 [==============================] - 10s 85us/step - loss: 4.2191e-04
Epoch 12/30
118740/118740 [==============================] - 10s 84us/step - los

In [39]:
y_pred_model2=model2.predict(X_test)
print("The mean absoulte error is :",mean_absolute_error(y_test,y_pred_model2))

The mean absoulte error is : 0.012631173459715548


Q.3

In [40]:
model3 = Sequential()
#Credits: https://gist.github.com/mustafa-qamaruddin
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]
def l2_regularization_param(X, y):
    btscv = BlockingTimeSeriesSplit(n_splits=5)
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {keras.regularizers.l2() : gammas}
    grid_search = GridSearchCV(model3.add(Dense(15, activation='relu', input_shape=(59,)), param_grid, cv=btscv)
    grid_search.model3.add(X,y)
    grid_search.best_param_
    return grid_search.best_param_
l2_regularization_param(X,y)

SyntaxError: invalid syntax (<ipython-input-40-4250f3b933d5>, line 26)

In [30]:
model3 = Sequential()
model3.add(Dense(15, activation='relu', input_shape=(59,),kernel_regularizer=keras.regularizers.l2(0.001)))
model3.add(Dense(10, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
model3.add(Dense(5, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
model3.add(Dense(1))

In [43]:
model3.compile(optimizer='adam', loss='mean_squared_error')

In [44]:
model3.fit(X,y,epochs=30, callbacks=[early_stopping_monitor],batch_size=20)

Epoch 1/30
118740/118740 [==============================] - 9s 79us/step - loss: 0.0013
Epoch 2/30
118740/118740 [==============================] - 9s 73us/step - loss: 4.2169e-04
Epoch 3/30
118740/118740 [==============================] - 8s 69us/step - loss: 4.2154e-04
Epoch 4/30
118740/118740 [==============================] - 8s 70us/step - loss: 4.2169e-04
Epoch 5/30
118740/118740 [==============================] - 8s 68us/step - loss: 4.2142e-04
Epoch 6/30
118740/118740 [==============================] - 8s 70us/step - loss: 4.2173e-04
Epoch 7/30
118740/118740 [==============================] - 9s 72us/step - loss: 4.2172e-04
Epoch 8/30
118740/118740 [==============================] - 7s 63us/step - loss: 4.2161e-04
Epoch 9/30
118740/118740 [==============================] - 9s 76us/step - loss: 4.2172e-04
Epoch 10/30
118740/118740 [==============================] - 8s 71us/step - loss: 4.2183e-04
Epoch 11/30
118740/118740 [==============================] - 8s 70us/step - loss: 4

In [46]:
y_pred_model3=model3.predict(X_test)
print("The mean absoulte error is :",mean_absolute_error(y_test,y_pred_model3))

The mean absoulte error is : 0.012742328417695684


Q.4

In [16]:
y_binary=y
y_test_binary=y_test

In [20]:
y_binary[y_binary<=0]=0
y_binary[y_binary>0]=1
y_test_binary[y_test_binary<=0]=0
y_test_binary[y_test_binary>0]=1

In [24]:
ohe = OneHotEncoder()
y_binary = ohe.fit_transform(np.array(y_binary).reshape(-1,1)).toarray()
y_test_binary=ohe.fit_transform(np.array(y_test_binary).reshape(-1,1)).toarray()

In [32]:
model4 = Sequential()
model4.add(Dense(15, activation='relu', input_shape=(59,)))
model4.add(layers.Dropout(0.3, input_shape=(59,)))
model4.add(Dense(10, activation='relu'))
model4.add(layers.Dropout(0.3, input_shape=(59,)))
model4.add(Dense(5, activation='relu'))
model4.add(layers.Dropout(0.3, input_shape=(59,)))
model4.add(Dense(2,activation='softmax'))

In [50]:
model4.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [51]:
model4.fit(X,y_binary, epochs=30, callbacks=[early_stopping_monitor],batch_size=20)

Epoch 1/30
118740/118740 [==============================] - 10s 82us/step - loss: 0.6901 - accuracy: 0.5313
Epoch 2/30
  1700/118740 [..............................] - ETA: 11s - loss: 0.6877 - accuracy: 0.5418

C:\Users\shahg_000\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


118740/118740 [==============================] - 10s 80us/step - loss: 0.6900 - accuracy: 0.5311
Epoch 3/30
118740/118740 [==============================] - 9s 79us/step - loss: 0.6900 - accuracy: 0.5323
Epoch 4/30
118740/118740 [==============================] - 9s 77us/step - loss: 0.6898 - accuracy: 0.5327
Epoch 5/30
118740/118740 [==============================] - 9s 79us/step - loss: 0.6898 - accuracy: 0.5307
Epoch 6/30
118740/118740 [==============================] - 9s 79us/step - loss: 0.6901 - accuracy: 0.5310
Epoch 7/30
118740/118740 [==============================] - 9s 77us/step - loss: 0.6899 - accuracy: 0.5311
Epoch 8/30
118740/118740 [==============================] - 9s 77us/step - loss: 0.6898 - accuracy: 0.5319
Epoch 9/30
118740/118740 [==============================] - 9s 80us/step - loss: 0.6897 - accuracy: 0.5323
Epoch 10/30
118740/118740 [==============================] - 9s 79us/step - loss: 0.6896 - accuracy: 0.5321
Epoch 11/30
118740/118740 [===================

In [53]:
y_pred_model4=model4.predict(X_test)

In [55]:
#Converting predictions to label
y_pred_convert = list()
for i in range(len(y_pred_model4)):
    y_pred_convert.append(np.argmax(y_pred_model4[i]))
#Converting one hot encoded test label to label
y_test_convert = list()
for i in range(len(y_test_binary)):
    y_test_convert.append(np.argmax(y_test_binary[i]))

In [56]:
print('The accuracy is:', accuracy_score(y_pred_convert,y_test_convert)*100)

The accuracy is: 49.60966954978931


END